In [1]:
import sndaq.util.legacy.sndata as snl
import numpy as np

In [2]:
user_live = 'REDACTED'
pass_live = 'REDACTED'
user_ldap = 'REDACTED'

In [3]:
run_info = snl.get_run_info(137104, USER=user_live, PASS=pass_live)
run_info

{'run_number': 137104,
 'status': 'SUCCESS',
 'config': 'sps-IC86-2022-DMIce-chargestamp-V309',
 'lightmode': 'dark',
 'run_mode': 'PhysicsTrig',
 'filter_mode': 'PhysicsFiltering',
 'pDAQ_release': 'Akupara5_16608:185361:',
 'live_start': '2022-09-30 18:04:31',
 'start': '2022-09-30 18:05:06',
 'start_frac': 4033675423,
 'precise_start_available': True,
 'stop': '2022-10-01 02:05:05',
 'stop_frac': 1136764904,
 'precise_stop_available': True,
 'grl_start': None,
 'grl_start_frac': None,
 'grl_stop': None,
 'grl_stop_frac': None,
 'switched_start': False,
 'latest_snapshot': None}

In [4]:
cand_live = snl.get_cands_from_live(137104, USER=user_live, PASS=pass_live)
cand_live[0]

{'_id': '633732b5129c8b',
 'service': 'sndaq',
 'varname': 'significance',
 'value': 5.33634,
 'prio': 1,
 'time': '2022-09-30 18:17:24.818526',
 'insert_time': '2022-09-30 18:17:35.852207'}

In [5]:
log_dir = './logs/'
cand_logs = snl.get_cands_from_log(137104, user_live, pass_live, user_ldap)
cand_logs[0]

{'time': numpy.datetime64('2022-09-30T18:06:47'),
 'xi': 5.33634,
 'ana': 1,
 'trigger': 1,
 'cand': 1}

In [6]:
sndata_file = './data/rundata/SPS/sndata_137104_000.root'
cand_data = snl.get_cands_from_sndata(sndata_file)
cand_data[0]

{'cand_no': 1,
 'Corr': <TGraph (version 4) at 0x7f55f4b0a4a8>,
 'DeCorr': <TGraph (version 4) at 0x7f55f4abae80>,
 'Corr_500': <TGraph (version 4) at 0x7f55f4abad30>,
 'DeCorr_500': <TGraph (version 4) at 0x7f55f4b489b0>}

In [7]:
def combine_cands(cand_live, cand_log, cand_data, *, tol=1e-5):
    try:
        assert(len(cand_live) == len(cand_log) == len(cand_data))
    except AssertionError:
        raise ValueError("Candidate sources must have equal lengths,"
                         f"given ({len(cand_live)} {len(cand_log)} {len(cand_data)}")
    data = []
    for i, (live, log, sndata) in enumerate(zip(cand_live, cand_log, cand_data)):
        xi_sndaq = sndata['Corr'].values()[1]
        diff_live = np.abs((log['xi'] - live['value'])/log['xi'])
        diff_sndaq = np.abs(xi_sndaq - log['xi']).min()/log['xi']
        
        try:
            assert(diff_live < tol and diff_sndaq < tol)
        except AssertionError:
            raise ValueError(f"Candidate {i:<3d} has differing xi greater than tolerance!")
        
        data.append({
            'trigger_time' : log['time'],
            'nh_alert_time' : live['time'],
            'xi' : xi_sndaq[np.abs(xi_sndaq - log['xi']).argmin()],
            'ana' : log['ana'],
            'trigger_no' : log['trigger'],
            'cand_no' : log['cand'],
        })
    return data

In [10]:
cands = combine_cands(cand_live, cand_logs, cand_data)
cands[0]

{'trigger_time': numpy.datetime64('2022-09-30T18:06:47'),
 'nh_alert_time': '2022-09-30 18:17:24.818526',
 'xi': 5.336336869863762,
 'ana': 1,
 'trigger_no': 1,
 'cand_no': 1}

In [9]:
import pickle
with open('cands_137104.pickle', 'wb') as f:
    pickle.dump(cands, f)